In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import os
from tqdm import tqdm
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import imblearn
from sklearn.calibration import calibration_curve,CalibratedClassifierCV
import seaborn as sns
import pandas as pd, numpy as np
import random
from sklearn.metrics import confusion_matrix, average_precision_score
import matplotlib.pyplot as plt 
%matplotlib inline
import re
import scipy.stats as st
from mord import LogisticIT, LogisticAT
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pprint import pprint
from sklearn import datasets, linear_model
from sklearn.metrics import confusion_matrix,f1_score,plot_confusion_matrix, classification_report, plot_roc_curve,accuracy_score, f1_score,roc_curve, roc_auc_score,mean_squared_error, mean_absolute_error,r2_score,precision_recall_curve
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression,Ridge,RidgeCV,LassoCV
from sklearn.model_selection import GridSearchCV, KFold,RepeatedKFold,train_test_split,RandomizedSearchCV,ParameterGrid
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [2]:
def clean_and_featurize(df):
    """Something is happening here to clean the data...."""
    labeled = df
    #labeled = labeled[['Report_ID','Report_Text','MLS_mm']]
    
    #rename columns to relevant
    labeled = labeled.rename(columns={"Report_ID": "Report_Number"})
    
    # Keep only those with IMPRESSIONS
    labeled = labeled.iloc[[x for x in range(labeled.shape[0]) if 'IMPRESSION:' in labeled.Report_Text.iloc[x]]]
    
    # replace whitespace with space ***************************
    labeled['Report_Text_clean'] = labeled['Report_Text'].apply(lambda text: ' '.join(text.split()))
    
    #REMOVE HEADER:
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: re.split('-'*78, text, 1)[-1])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: re.split('HISTORY:', text, 1)[-1])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: re.split('REPORT ', text, 1)[-1])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: re.split('REPORT:', text, 1)[-1])
    
    #REMOVE FOOTER:
    # Remove footer parts
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       re.split('electronically signed by:', text, flags=re.IGNORECASE)[0])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       ''.join(re.split('i, the teaching physician, have reviewed the images and agree with the report as written', text, flags=re.IGNORECASE)))
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       re.split('radiologists: signatures:', text, flags=re.IGNORECASE)[0])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       re.split('providers: signatures:', text, flags=re.IGNORECASE)[0])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       re.split('findings were discussed on', text, flags=re.IGNORECASE)[0])
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: 
                                                       re.split('this report was electronically signed by', text, flags=re.IGNORECASE)[0])

    # Remove reference texts =====
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: ''.join([x for i,x in enumerate(text.split('='*34)) if i != 1]))
    
    labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: text.lower())
    
    
    # Replace ngrams in Report_Text & IMPRESSION with their units
    for group in wordgroups:
        labeled['Report_Text_clean'] = labeled.Report_Text_clean.apply(lambda text: text.replace(group, ''.join(group.split())))

    #labeled = labeled.drop_duplicates(subset=['Report_Number'])
    labeled = labeled.reset_index(drop=True)
    
    return labeled

In [3]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), st.sem(a)
    h = se * st.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [4]:
def create_externalValidation_BOW_matrix(external_val_df,fit_df,vectorizer='count',doc_col_name="Report_Text_clean"):
    """Trains a vectorizer on the FIT DATASET and then applies it to an EXTERNAL VALIDATION set. The
    result is once more a matrix in which entries are token counts per document. Only returns this matrix
    as a dataframe for the EXTERNAL VALIDATION set; at present, the 
    
    :params
    1) external_val_df: the preprocessed dataframe of stroke reports from your external validation dataset
    2) fit_df: the preprocessed data for the FITTING dataset (MGB large acute strokes)
    3) vectorizer: determines count or TF-IDF vectorization"""
    #initialize vectorizer
    if vectorizer=='count':
        vectorizer = CountVectorizer(min_df=0.01) #ignore if term appears in <cutoff % of documents
    else:
        vectorizer=TfidfVectorizer(min_df=0.01)
        
        
    #1) fit the vectorizer on the training data
    #Obtain counts using the column name corresponding to the radiology reports
    vectorizer.fit(external_val_df[doc_col_name])
    
    #
    fit_inTermsof_extVal=vectorizer.transform(fit_df[doc_col_name]).todense() #now transform train dataset in terms of test features
    extVal_inTermsof_extVal=vectorizer.transform(external_val_df[doc_col_name]).todense()
    
    
    
    #3) Obtain feature names for dataframe columns
    feature_names=vectorizer.get_feature_names()
    
    #this is the train set outcomes in terms of the test set features
    X_fit=pd.DataFrame(fit_inTermsof_extVal,columns=feature_names)
    X_extVal=pd.DataFrame(extVal_inTermsof_extVal,columns=feature_names)
    
    return X_fit,X_extVal

In [5]:
def extVal_classifier_complete(n_trials,cleaned_fit_df,cleaned_extVal_df,fit_BOW_matrix,extVal_BOW_matrix,doc_col_name,classifier,print_complete=True):
    """A comprehensive function that takes in all data and a classifier of choice then computes 
    epidemiologic statistics for that classifier as well as returns numpy arrays corresponding to 
    ROC, PR, and calibration curves. Hyperparameter tuning is included within classifier definitions, which
    occur within the body of the function as below"""
    
    num_trials = n_trials
    score = 'roc_auc' #metric by which to evaluate
    
    ###########GRAPHS###########################
    
    #ROC
    fprs = []
    tprs = []
    
    #PR
    
    precisions_4graph = [] #this vector is distinct from the epdemiologic measures of "precision"
    recalls = []
    
    #Calibration curves
    mpvs = []
    fops = []
    
    
    #########STATS##############################
    #statistics
    accuracies=[]
    aucs=[]
    auprs=[]
    specificities=[]
    sensitivities=[]
    precisions=[]
    f1s=[]
    
    
    #STATS
    accuracy=0
    aupr=0
    auc = 0 #will keep track of average AUC at the end of this all
    specificity=0
    sensitivity=0
    precision=0
    f1=0
    
    sm = SMOTE(random_state=42)
    #oversample = imblearn.RandomOverSampler(sampling_strategy='minority')
    
    for trial in range(1,num_trials+1):
        print('Results for trial ',trial)
        X_train,_,y_train,_ = train_test_split(fit_BOW_matrix,cleaned_fit_df[str(doc_col_name)],test_size=0.20)
        
        X_res, y_res = sm.fit_resample(X_train, y_train)
        
        if classifier=='logreg':
        
            tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]

            #initialize Logistic Regression with GridSearch over 'tolerance' parameter
            outcome_presence = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100, solver='liblinear'), 
                                        tuned_parameters, cv = 10, scoring = 'roc_auc')
            
                    #Calibrate the classifier with Platt scaling
            outcome_presence=CalibratedClassifierCV(outcome_presence)
            
            
        elif classifier=='rf':
            
            maxTrees = 200
            maxDepth = 10
            minBucket = 10
            
            tuned_parameters = [{'n_estimators': range(10,maxTrees+10,100),'max_depth':range(3,maxDepth+1), 'min_samples_leaf':range(1,minBucket+1), 'bootstrap' : [True, False]}]
        
            outcome_presence = RandomizedSearchCV(RandomForestClassifier(criterion="gini", random_state=100),
                               tuned_parameters, cv=10, scoring = score) 
            
            outcome_presence=CalibratedClassifierCV(outcome_presence)
            
        elif classifier=='nn':
            mlp_parameters = [{'hidden_layer_sizes': [50,100,200]}]
        
        #initialize Logistic Regression with GridSearch over 'tolerance' parameter
            outcome_presence = GridSearchCV(MLPClassifier(learning_rate_init=0.003), 
                                        mlp_parameters, cv = 5, scoring = 'roc_auc')
            
        elif classifier=='CART':
            maxDepth = 10
            minBucket = 10
            tuned_parameters = [{'max_depth':range(3,maxDepth+1), 'min_samples_leaf':range(1,minBucket+1)}]
            outcome_presence = GridSearchCV(tree.DecisionTreeClassifier(criterion="gini", random_state=100, splitter = 'best'),
                               tuned_parameters, cv=10, scoring = score)
        
            #Calibrate the classifier with Platt scaling
            outcome_presence=CalibratedClassifierCV(outcome_presence)
            
        elif classifier=='svm':
            tuned_parameters = {'C': [0.1,], 
                          'gamma': [0.1, 0.01, 0.001]} 
            outcome_presence=GridSearchCV(SVC(),tuned_parameters,cv=5,scoring=score)
            outcome_presence=CalibratedClassifierCV(outcome_presence)
            
        elif classifier=='knn':
            max_neighbors=7
            tuned_parameters=[{'n_neighbors':range(2,max_neighbors)}]
            outcome_presence = GridSearchCV(KNeighborsClassifier(),
                   tuned_parameters, cv=10, scoring = score)
            outcome_presence=CalibratedClassifierCV(outcome_presence)
            
        
        #SMOTE resampling
        #outcome_presence.fit(X_train,y_train)
        
        #external testing data 
        X_test,y_test=extVal_BOW_matrix,cleaned_extVal_df[str(doc_col_name)]
        
        outcome_presence.fit(X_res, y_res)
        
        #categorical predictions
        y_pred = outcome_presence.predict(X_test)
        #raw probabilities
        y_proba = outcome_presence.predict_proba(X_test)[:,1]
        
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        print(type(y_pred))
        
        if print_complete:
            print(f'TP: {tp} \n')
            print(f'TN: {tn} \n')
            print(f'FP: {fp} \n')
            print(f'FN: {fn} \n')
            print(3*'\n')

                
        #for optional plotting:
        fpr,tpr,_ = roc_curve(y_test,y_proba) #ROC
        pr_4graph,recall,_ = precision_recall_curve(y_test,y_proba) #PR
        fop, mpv = calibration_curve(y_test, y_proba, n_bins=10) #calibration

        
        #ROC Curve
        fprs.append(fpr)
        tprs.append(tpr)
        
        #PR Curve
        precisions_4graph.append(pr_4graph)
        recalls.append(recall)
        
        #Calibration Curve
        mpvs.append(mpv)
        fops.append(fop)
        
        
        
        #print(roc_auc_score(y_test,y_proba))
        
        #for specific trial
        trial_auc=roc_auc_score(y_test, y_proba)
        trial_aupr=average_precision_score(y_test,y_proba)
        trial_sensitivity=tp/(tp+fn)
        trial_specificity=tn/(tn+fp)
        trial_precision=tp/(tp+fp)
        trial_f1=f1_score(y_test,y_pred)
        trial_accuracy=(tp+tn)/(tp+tn+fp+fn)
        
        #for averaging over all trials
        auc += trial_auc #update running sum of AUC scores
        sensitivity+=trial_sensitivity
        aupr+=trial_aupr
        specificity+=trial_specificity
        precision+=trial_precision
        f1+=trial_f1
        accuracy+=trial_accuracy
        
        #for standard dev over all trials 
        aucs.append(trial_auc)
        sensitivities.append(trial_sensitivity)
        auprs.append(trial_aupr)
        specificities.append(trial_specificity)
        precisions.append(trial_precision)
        f1s.append(trial_f1)
        accuracies.append(trial_accuracy)
        
        
        
    #average values 
    average_AUC = auc/n_trials
    average_AUPR=aupr/n_trials
    average_sensitivity=sensitivity/n_trials
    average_specificity=specificity/n_trials
    average_precision=precision/n_trials
    average_f1=f1/n_trials
    average_accuracy=accuracy/n_trials
    
    
    #standard deviations
#     std_auc=np.round(np.std(aucs),3)
#     std_sens=np.round(np.std(sensitivities),3)
#     std_spec=np.round(np.std(specificities),3)
#     std_prec=np.round(np.std(precisions),3)
#     std_f1=np.round(np.std(f1s),3)
#     std_acc=np.round(np.std(accuracies),3)
    
    #95th %ile Confidence Intervals:
    ci95_auc=mean_confidence_interval(aucs)
    ci95_aupr=mean_confidence_interval(auprs)
    ci95_sens=mean_confidence_interval(sensitivities)
    ci95_spec=mean_confidence_interval(specificities)
    ci95_prec=mean_confidence_interval(precisions)
    ci95_acc=mean_confidence_interval(accuracies)
    
    
    

    print(f'{classifier} for {doc_col_name}.\n ======================================\n')
    
    print(f'AUC [{ci95_auc}]. AUPR [{ci95_aupr}]. Sensitivity [{ci95_sens}]. Specificity[{ci95_spec}]\n')
    print(f'Precision [{ci95_prec}]. Accuracy [{ci95_acc}]')

    ###RETURN THE ARRAYS FOR THE GRAPHS
    return fprs,tprs,precisions_4graph,recalls,mpvs,fops

In [20]:
# def extVal_classifier_complete(n_trials,cleaned_fit_df,cleaned_extVal_df,fit_BOW_matrix,extVal_BOW_matrix,doc_col_name,classifier,print_complete=True):
#     """A comprehensive function that takes in all data and a classifier of choice then computes 
#     epidemiologic statistics for that classifier as well as returns numpy arrays corresponding to 
#     ROC, PR, and calibration curves. Hyperparameter tuning is included within classifier definitions, which
#     occur within the body of the function as below"""
    
#     num_trials = n_trials
#     score = 'roc_auc' #metric by which to evaluate
    
#     ###########GRAPHS###########################
    
#     #ROC
#     fprs = []
#     tprs = []
    
#     #PR
    
#     precisions_4graph = [] #this vector is distinct from the epdemiologic measures of "precision"
#     recalls = []
    
#     #Calibration curves
#     mpvs = []
#     fops = []
    
    
#     #########STATS##############################
#     #statistics
#     accuracies=[]
#     aucs=[]
#     specificities=[]
#     sensitivities=[]
#     precisions=[]
#     f1s=[]
    
    
#     #STATS
#     accuracy=0
#     auc = 0 #will keep track of average AUC at the end of this all
#     specificity=0
#     sensitivity=0
#     precision=0
#     f1=0
    
#     sm = SMOTE(random_state=42)
    
#     for trial in range(1,num_trials+1):
#         print('Results for trial ',trial)
#         X_train,_,y_train,_ = train_test_split(fit_BOW_matrix,cleaned_fit_df[str(doc_col_name)],test_size=0.20)
        
#         X_res, y_res = sm.fit_resample(X_train, y_train)
        
#         if classifier=='logreg':
        
#             tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]

#             #initialize Logistic Regression with GridSearch over 'tolerance' parameter
#             outcome_presence = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100, solver='liblinear'), 
#                                         tuned_parameters, cv = 10, scoring = 'roc_auc')
            
#                     #Calibrate the classifier with Platt scaling
#             outcome_presence=CalibratedClassifierCV(outcome_presence)
            
            
#         elif classifier=='rf':
            
#             maxTrees = 200
#             maxDepth = 10
#             minBucket = 10
            
#             tuned_parameters = [{'n_estimators': range(10,maxTrees+10,100),'max_depth':range(3,maxDepth+1), 'min_samples_leaf':range(1,minBucket+1), 'bootstrap' : [True, False]}]
        
#             outcome_presence = RandomizedSearchCV(RandomForestClassifier(criterion="gini", random_state=100),
#                                tuned_parameters, cv=10, scoring = score) 
            
#             outcome_presence=CalibratedClassifierCV(outcome_presence)
            
#         elif classifier=='nn':
#             mlp_parameters = [{'hidden_layer_sizes': [50,100,200]}]
        
#         #initialize Logistic Regression with GridSearch over 'tolerance' parameter
#             outcome_presence = GridSearchCV(MLPClassifier(learning_rate_init=0.003), 
#                                         mlp_parameters, cv = 5, scoring = 'roc_auc')
            
#         elif classifier=='CART':
#             maxDepth = 10
#             minBucket = 10
#             tuned_parameters = [{'max_depth':range(3,maxDepth+1), 'min_samples_leaf':range(1,minBucket+1)}]
#             outcome_presence = GridSearchCV(tree.DecisionTreeClassifier(criterion="gini", random_state=100, splitter = 'best'),
#                                tuned_parameters, cv=10, scoring = score)
        
#             #Calibrate the classifier with Platt scaling
#             outcome_presence=CalibratedClassifierCV(outcome_presence)
            
#         elif classifier=='svm':
#             tuned_parameters = {'C': [0.1,], 
#                           'gamma': [0.1, 0.01, 0.001]} 
#             outcome_presence=GridSearchCV(SVC(),tuned_parameters,cv=5,scoring=score)
#             outcome_presence=CalibratedClassifierCV(outcome_presence)
            
#         elif classifier=='knn':
#             max_neighbors=7
#             tuned_parameters=[{'n_neighbors':range(2,max_neighbors)}]
#             outcome_presence = GridSearchCV(KNeighborsClassifier(),
#                    tuned_parameters, cv=10, scoring = score)
#             outcome_presence=CalibratedClassifierCV(outcome_presence)
            
        
#         #SMOTE resampling
#         outcome_presence.fit(X_res,y_res)
        
#         #external testing data 
#         X_test,y_test=extVal_BOW_matrix,cleaned_extVal_df[str(doc_col_name)]
        
#         #outcome_presence.fit(X_train, y_train)
        
#         #categorical predictions
#         y_pred = outcome_presence.predict(X_test)
#         #raw probabilities
#         y_proba = outcome_presence.predict_proba(X_test)[:,1]
        
#         tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
#         print(type(y_pred))
        
#         if print_complete:
#             print(f'TP: {tp} \n')
#             print(f'TN: {tn} \n')
#             print(f'FP: {fp} \n')
#             print(f'FN: {fn} \n')
#             print(3*'\n')

                
#         #for optional plotting:
#         fpr,tpr,_ = roc_curve(y_test,y_proba) #ROC
#         pr_4graph,recall,_ = precision_recall_curve(y_test,y_proba) #PR
#         fop, mpv = calibration_curve(y_test, y_proba, n_bins=10) #calibration

        
#         #ROC Curve
#         fprs.append(fpr)
#         tprs.append(tpr)
        
#         #PR Curve
#         precisions_4graph.append(pr_4graph)
#         recalls.append(recall)
        
#         #Calibration Curve
#         mpvs.append(mpv)
#         fops.append(fop)
        
        
        
#         #print(roc_auc_score(y_test,y_proba))
        
#         #for specific trial
#         trial_auc=roc_auc_score(y_test, y_proba)
#         trial_sensitivity=tp/(tp+fn)
#         trial_specificity=tn/(tn+fp)
#         trial_precision=tp/(tp+fp)
#         trial_f1=f1_score(y_test,y_pred)
#         trial_accuracy=(tp+tn)/(tp+tn+fp+fn)
        
#         #for averaging over all trials
#         auc += trial_auc #update running sum of AUC scores
#         sensitivity+=trial_sensitivity
#         specificity+=trial_specificity
#         precision+=trial_precision
#         f1+=trial_f1
#         accuracy+=trial_accuracy
        
#         #for standard dev over all trials 
#         aucs.append(trial_auc)
#         sensitivities.append(trial_sensitivity)
#         specificities.append(trial_specificity)
#         precisions.append(trial_precision)
#         f1s.append(trial_f1)
#         accuracies.append(trial_accuracy)
        
        
        
#     #average values 
#     average_AUC = auc/n_trials
#     average_sensitivity=sensitivity/n_trials
#     average_specificity=specificity/n_trials
#     average_precision=precision/n_trials
#     average_f1=f1/n_trials
#     average_accuracy=accuracy/n_trials
    
    
#     #standard deviations
# #     std_auc=np.round(np.std(aucs),3)
# #     std_sens=np.round(np.std(sensitivities),3)
# #     std_spec=np.round(np.std(specificities),3)
# #     std_prec=np.round(np.std(precisions),3)
# #     std_f1=np.round(np.std(f1s),3)
# #     std_acc=np.round(np.std(accuracies),3)
    
#     #95th %ile Confidence Intervals:
#     ci95_auc=st.t.interval(alpha=0.95, df=len(aucs)-1, loc=np.mean(aucs), scale=st.sem(aucs))
#     ci95_sens=st.t.interval(alpha=0.95, df=len(sensitivities)-1, loc=np.mean(sensitivities), scale=st.sem(sensitivities))
#     ci95_spec=st.t.interval(alpha=0.95, df=len(specificities)-1, loc=np.mean(specificities), scale=st.sem(specificities))
#     ci95_prec=st.t.interval(alpha=0.95, df=len(precisions)-1, loc=np.mean(precisions), scale=st.sem(precisions))
#     ci95_f1=st.t.interval(alpha=0.95, df=len(f1s)-1, loc=np.mean(f1s), scale=st.sem(f1s))
#     ci95_acc=st.t.interval(alpha=0.95, df=len(accuracies)-1, loc=np.mean(accuracies), scale=st.sem(accuracies))
    
    
    
#     print(f'{classifier} for {doc_col_name}.\n ======================================\n')
#     print(f'AUC {average_AUC} [{ci95_auc}]. Sensitivity {average_sensitivity} [{ci95_sens}]. Specificity {average_specificity} [{ci95_spec}]\n')
#     print(f'Precision {average_precision} [{ci95_prec}]. F1 {average_f1} [{ci95_f1}]. Accuracy {average_accuracy} [{ci95_acc}]')
    

#     ###RETURN THE ARRAYS FOR THE GRAPHS
#     return fprs,tprs,precisions_4graph,recalls,mpvs,fops

### Prepare Data 0: Fit DF (BWH Large Stroke)

In [6]:
df = pd.read_csv('6.1.21 deidComplete REDCap Reports Non-duplicate Version 3.csv')
df.rename(columns={'report_text': 'Report_Text', 'mls_mm_v2': 'MLS_mm','record_id':'Report_ID','ecass_v2':'ECASS','edema_severity_report':'edema_severity'}, inplace=True)

df=df.dropna(subset=['ivh_v2'])
df['ivh_present']=df['ivh_v2'].apply(lambda x: 0 if x==1.0 else 1)
df['ECASS']=df['ECASS'].fillna(0)
df['severe_ecass']=df['ECASS'].apply(lambda x: 0 if x<3.0 else 1)
df['MLS_mm']=df['MLS_mm'].fillna(0)
df['MLS_presence']=df['MLS_mm'].apply(lambda x: 0 if x==0 else 1)
#df_edema = df[np.logical_or(df['MLS_presence']==0,df['MLS_presence']==1)]
df['edema_report']=df['edema_report'].fillna(0)
df['edema_report']=df['edema_report'].apply(lambda x: int(x))

wordgroups = list(pd.read_excel('wordgroups.xlsx')['Word Groupings'])#now clean and featurize
fit_df_overall = clean_and_featurize(df)

### Prepare Data 1: BMC Below

In [7]:
tr=pd.read_csv('OCTOBER8_377_BMC_ACUTE_STROKE_CONFIRMED.csv')

In [8]:
clean_extVal_df = pd.read_csv('OCTOBER8_377_BMC_ACUTE_STROKE_CONFIRMED.csv')


clean_extVal_df.rename(columns={'ecass':'ECASS','ivh':'ivh_present','edema':'edema_report','hemcon':'hem_conv_v2','midline_shift_mm':'MLS_mm'}, inplace=True)

# clean_extVal_df=clean_extVal_df.dropna(subset=['ivh_v2'])
# clean_extVal_df['ivh_present']=clean_extVal_df['ivh_v2'].apply(lambda x: 0 if x==1.0 else 1)
clean_extVal_df['ECASS']=clean_extVal_df['ECASS'].fillna(0)
clean_extVal_df['severe_ecass']=clean_extVal_df['ECASS'].apply(lambda x: 0 if x<3.0 else 1)
#clean_extVal_df['MLS_mm']=clean_extVal_df['MLS_mm'].fillna(0)

clean_extVal_df['MLS_presence']=clean_extVal_df['MLS_mm'].apply(lambda x: 0 if np.logical_or(x==0,x==999) else 1)
# #clean_extVal_df_edema = clean_extVal_df[np.logical_or(clean_extVal_df['MLS_presence']==0,clean_extVal_df['MLS_presence']==1)]
clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].fillna(0)
clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].apply(lambda x: int(x))

wordgroups = list(pd.read_excel('wordgroups.xlsx')['Word Groupings'])#now clean and featurize
clean_extVal_df = clean_and_featurize(clean_extVal_df)

In [7]:
# clean_extVal_df=pd.read_csv('bwh_strokes_not_just_large_v2.csv')
# clean_extVal_df=clean_extVal_df.rename(columns={'Report_Text':'Report_Text_clean','mls_mm_v2':'MLS_mm','ecass_v2':'ECASS'})
# clean_extVal_df=clean_extVal_df.dropna(subset=['ivh_v2'])
# clean_extVal_df['ivh_present']=clean_extVal_df['ivh_v2'].apply(lambda x: 0 if x==1.0 else 1)
# clean_extVal_df['ECASS']=clean_extVal_df['ECASS'].fillna(0)
# clean_extVal_df['severe_ecass']=clean_extVal_df['ECASS'].apply(lambda x: 0 if x<3.0 else 1)
# clean_extVal_df['MLS_mm']=clean_extVal_df['MLS_mm'].fillna(0)
# clean_extVal_df['MLS_presence']=clean_extVal_df['MLS_mm'].apply(lambda x: 0 if x==0 else 1)
# #clean_extVal_df_edema = clean_extVal_df[np.logical_or(clean_extVal_df['MLS_presence']==0,clean_extVal_df['MLS_presence']==1)]
# clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].fillna(0)
# clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].apply(lambda x: int(x))

## Prepare MGB Non-Large Data Below:

In [13]:
mgb_nonlarge1=pd.read_csv('OCTOBER15_BWH_NONLARGE_BATCH1.csv')
mgb_nonlarge2=pd.read_csv('OCTOBER15_BWH_NONLARGE_BATCH2.csv')


In [14]:
mgb_nonlarge2['Unprocessed_Report_Text']

0      Exam Number:  E1082208                        ...
1      Exam Number:  16915447                        ...
2      Exam Number:  17732643                        ...
3      Exam Number:  A13814506                       ...
4      Exam Number:  17173917                        ...
                             ...                        
332    Exam Number:  18223378                        ...
333    Exam Number:  E1300108                        ...
334    Exam Number:  17081936                        ...
335    Exam Number:  18202901                        ...
336    Exam Number:  16821557                        ...
Name: Unprocessed_Report_Text, Length: 337, dtype: object

In [15]:
mgb_nonlarge1=pd.read_csv('OCTOBER15_BWH_NONLARGE_BATCH1.csv')
mgb_nonlarge2=pd.read_csv('OCTOBER15_BWH_NONLARGE_BATCH2.csv')
mgb_nonlarge2=mgb_nonlarge2[['Report Number','Unprocessed_Report_Text','EMPI','Report_Description','Age','Gender','edema_report','mls_mm_v2','hem_conv_v2','ecass_v2','ivh_v2']]
mgb_nonlarge1=mgb_nonlarge1[['Report_Number','Unprocessed_Report_Text','EMPI','Report_Description','Age','Gender','edema_report','mls_mm_v2','hem_conv_v2','ecass_v2','ivh_v2']]
clean_extVal_df=pd.concat([mgb_nonlarge1,mgb_nonlarge2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [17]:
clean_extVal_df.rename(columns={'Unprocessed_Report_Text':'Report_Text','ecass_v2':'ECASS','mls_mm_v2':'MLS_mm'}, inplace=True)

clean_extVal_df=clean_extVal_df.dropna(subset=['ivh_v2'])
clean_extVal_df['ivh_present']=clean_extVal_df['ivh_v2'].apply(lambda x: 0 if x==1.0 else 1)
clean_extVal_df['ECASS']=clean_extVal_df['ECASS'].fillna(0)
clean_extVal_df['severe_ecass']=clean_extVal_df['ECASS'].apply(lambda x: 0 if x<3.0 else 1)

clean_extVal_df['MLS_mm']=clean_extVal_df['MLS_mm'].fillna(0)

clean_extVal_df['MLS_presence']=clean_extVal_df['MLS_mm'].apply(lambda x: 0 if np.logical_or(x==0,x==999) else 1)
# #clean_extVal_df_edema = clean_extVal_df[np.logical_or(clean_extVal_df['MLS_presence']==0,clean_extVal_df['MLS_presence']==1)]
clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].fillna(0)
clean_extVal_df['edema_report']=clean_extVal_df['edema_report'].apply(lambda x: int(x))

wordgroups = list(pd.read_excel('wordgroups.xlsx')['Word Groupings'])#now clean and featurize


In [18]:
clean_extVal_df = clean_and_featurize(clean_extVal_df)

### #RunIt

In [19]:
for classifier in ['logreg']:#,'knn','rf','nn']:
#for classifier in ['logreg']:
    #for outcome in ['ivh_present','hem_conv_v2','severe_ecass','edema_report','MLS_presence','hem_conv_v2',]:

    #for outcome in ['severe_ecass']:
    for outcome in ['ivh_present']: #hem_conv_v2','severe_ecass','ivh_present']:
    #for outcome in ['edema_report']:
    
        for vectorizer in ['count','tfidf']:

            print(f'Now testing {classifier} with {outcome} and {vectorizer} vectorizer')


            fit_df=fit_df_overall[np.logical_or(fit_df_overall[outcome]==0,fit_df_overall[outcome]==1)]
            clean_extVal_df=clean_extVal_df[np.logical_or(clean_extVal_df[outcome]==0,clean_extVal_df[outcome]==1)]
            X_fit,X_extval=create_externalValidation_BOW_matrix(clean_extVal_df,fit_df,vectorizer=vectorizer)

            
            
            #Run Unique Classifier
            fprs,tprs,precisions,recalls,mpvs,fops=extVal_classifier_complete(3,fit_df,clean_extVal_df,X_fit,X_extval,outcome,classifier,print_complete=True)
            
            
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_fprs.npy',fprs)
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_tprs.npy',tprs)
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_precisions.npy',precisions)
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_recalls.npy',recalls)
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_mpvs.npy',mpvs)
            np.save(f'numpys/MGB_nonLarge_{classifier}_{vectorizer}_{outcome}_fops.npy',fops)




Now testing logreg with ivh_present and count vectorizer
Results for trial  1
<class 'numpy.ndarray'>
TP: 7 

TN: 490 

FP: 10 

FN: 11 





Results for trial  2
<class 'numpy.ndarray'>
TP: 9 

TN: 490 

FP: 10 

FN: 9 





Results for trial  3
<class 'numpy.ndarray'>
TP: 11 

TN: 492 

FP: 8 

FN: 7 





logreg for ivh_present.

AUC [(0.9045925925925927, 0.8746397390138012, 0.9345454461713842)]. AUPR [(0.5256007918949691, 0.39309732634283756, 0.6581042574471007)]. Sensitivity [(0.5, 0.22398469868402737, 0.7760153013159726)]. Specificity[(0.9813333333333333, 0.9755964630267849, 0.9870702036398816)]

Precision [(0.48813209494324045, 0.27816636416137014, 0.6980978257251107)]. Accuracy [(0.9646074646074645, 0.9499565648632967, 0.9792583643516323)]
Now testing logreg with ivh_present and tfidf vectorizer
Results for trial  1
<class 'numpy.ndarray'>
TP: 17 

TN: 488 

FP: 12 

FN: 1 





Results for trial  2
<class 'numpy.ndarray'>
TP: 17 

TN: 498 

FP: 2 

FN: 1 





Results for tria

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('OCTOBER1_BMC_SPREADSHEET_WITH_DEMOGRAPHICS.csv')

In [6]:
df_last=df.iloc[-350:,:]

In [7]:
df_last.to_csv('OCTOBER3_BMC_SPREADSHEET_LAST_ROWS.csv')